In [2]:
from coco_utils import load_coco_info, load_coco_image
import numpy as np
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
from collections import OrderedDict
import json
import random
import os
from PIL import Image, ImageOps
import plotly.express as px
import pickle

In [3]:
coco_instances = COCO("dataset/annotations/instances_train2017.json")

loading annotations into memory...
Done (t=21.88s)
creating index...
index created!


In [4]:
coco_stuff = COCO("dataset/annotations/stuff_train2017.json")

loading annotations into memory...
Done (t=43.34s)
creating index...
index created!


In [9]:
coco_captions = load_coco_info("dataset/annotations/captions_train2017.json")

In [10]:
# they all have the same # of images, but the available data varies
print(len(coco_captions['images']), len(coco_instances.dataset['images']), len(coco_stuff.dataset['images']))
print(len(coco_captions['annotations']), len(coco_instances.dataset['annotations']), len(coco_stuff.dataset['annotations']))

118287 118287 118287
591753 860001 747458


In [11]:
avail_caption_imgs = sorted(list(set(([key['image_id'] for key in coco_captions['annotations']]))))
avail_instance_imgs = sorted(list(set(([key['image_id'] for key in coco_instances.dataset['annotations']]))))
avail_stuff_imgs = sorted(list(set(([key['image_id'] for key in coco_stuff.dataset['annotations']]))))

In [12]:
valid_imgs = sorted(set.intersection(*map(set,[avail_caption_imgs, avail_instance_imgs, avail_stuff_imgs])))
coco_images = {}
sorted_stuff = {}

for ann in coco_stuff.dataset['annotations']:
  img_id = ann['image_id']
  if img_id in sorted_stuff.keys():
    sorted_stuff[img_id].append(ann)
  else:
    sorted_stuff[img_id] = []

sorted_instances = {}

for instance in coco_instances.dataset['annotations']:
  img_id = instance["image_id"]
  if instance["image_id"] in sorted_instances.keys():
    sorted_instances[img_id].append(instance)
  else:
    sorted_instances[img_id] = []

sorted_captions = {}
for cap in coco_captions['annotations']:
    sorted_captions[cap['image_id']] = cap

licenses = {}
for l in coco_captions['licenses']:
    licenses[l['id']] = l

sorted_images = {}
for img in coco_captions['images']:
  sorted_images[img['id']] = img

for img_id in valid_imgs:

  image = sorted_images[img_id]

  annotation = sorted_captions[img_id]

  instance_ann = sorted_instances[img_id]

  stuff_ann = sorted_stuff[img_id]

  coco_images[img_id] = {
      "filename" : image['file_name'],
      "dims" : (image['height'], image['width']),
      "date" : image["date_captured"],
      "license_id" : int(licenses[image['license']]['id']),
      "license_name" : licenses[image['license']]['name'],
      "ann_id" : image['id'],
      "caption" : annotation["caption"],
      "caption_id" : annotation["id"],
      "instance_ann" : instance_ann,
      "stuff_ann" : stuff_ann,
  }


In [13]:
with open('dataset/coco_organized.pickle', 'wb') as handle:
    pickle.dump(coco_images, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open('dataset/coco_organized.pickle', 'rb') as handle:
    coco_images = pickle.load(handle)

In [5]:
category_map = {}

for cat in coco_stuff.dataset['categories']:
  category_map[cat['id']] = cat

for cat in coco_instances.dataset['categories']:
  category_map[cat['id']] = cat

In [6]:
with open('dataset/category_map.pickle', 'wb') as handle:
    pickle.dump(category_map, handle, protocol=pickle.HIGHEST_PROTOCOL)